<a href="https://colab.research.google.com/github/nathan-silveira/GEMINI-ANALISE-SENTIMENTOS/blob/main/PLN_PROJETO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJETO PLN - ANÁLISE DE SENTIMENTOS DE NOTÍCIAS POLÍTICAS**
Prof. Alexandre Donizeti Alves

### **EQUIPE**
---

**Integrante**

`Nathan Silveira Alves RA: 11201920933`

### **O MODELO LLM**

---

>


**LLM**: GEMINI 2.5-FLASH

>

**Link para a documentação oficial**: https://ai.google.dev/gemini-api/docs/models?hl=pt-br#gemini-2.5-flash

### **API**
---

**API**: NewsAPI

**Site oficial**: https://newsapi.org/

**Link para a documentação oficial**: https://newsapi.org/docs

### **DESCRIÇÃO**
---

O projeto consiste em basicamente duas etapas:
- obtenção das notícias do NewsAPI, seguindo alguns critérios;
- utilização da LLM para aplicação das técnicas e análise posterior dos resultados com assertividade.







#### Obtenção das notícias
Em relação as notícias, o primeiro pontoa a se destacar é que foi escolhido o idioma e, por consequência, os veículos de comunicação em linguagem inglesa, pois está presente mais veículos polarizados, o que será vantajoso para as aplicações da LLM e sua análise posterior.

Além deste, os parâmetros utilizados para obter as notícias foram:
- query = "Trump": para ter certeza que a notícia será política;
- domains = "foxnews.com,bbc.co.uk,msnbc.com": A escolha destes foi baseado em polaridade política, isto é, a FOX News foi a escolhida para representar um veículo de comunicação de direita, o MSNBC para representar de esquerda e a BBC para representar um neutro;
- sortBy = publishedAt: para fugir de qualquer viés que os artigos mais populares poderiam trazer;
- pageSize = 30: Em cada execução, apenas 30 notícias de cada domínio foi salva no corpus, em cada execução. Isto para evitar ultrapassar o limite de 250 requisições por dia da LLM.

Após isso, foram salvos no corpus os seguintes campos:
- title: o título da notícia;
- text: o resumo da notícia;
- source: o veículo de comunicação responsável pela notícia;
- url: o endereço eletrônico da notícia.

#### Uso da LLM

A LLM foi utilizada para aplicar 3 técnicas:
- Análise de sentimentos;
- Extrair 5 palavras chaves;
- Reconhecer entidades nomeadas.




### **IMPLEMENTAÇÃO**
---

In [ ]:
!pip install langchain -q U
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [ ]:
import os
import requests
import pandas as pd
from getpass import getpass

# Configuração da chave da NEWSAPI e criação do arquivo de saída
RESULT_FILE_NAME = "analise_de_viés_agregada.csv"

API_KEY = os.environ.get("NEWS_API_KEY") 

if not API_KEY:
    API_KEY = getpass("Digite sua Chave da News API: ")


Digite sua Chave da News API: ··········


In [ ]:
# função para coletar as notícias que serão usadas posteriormente para análise

def get_news(query):
    if "NEWS_API_KEY" not in os.environ:
        os.environ["NEWS_API_KEY"] = getpass("Digite sua Chave da News API: ")

    NEWS_API_KEY = os.environ.get("NEWS_API_KEY")

    DOMAINS = "foxnews.com,bbc.co.uk,msnbc.com"
    corpus = []

    for dominio in DOMAINS.split(','):
        url = ('https://newsapi.org/v2/everything?'
        'q='+ query + '&'
        'domains=' + dominio +'&'
        'language=en&'
        'sortBy=publishedAt&'
        'pageSize=30&'
        'apiKey=' + NEWS_API_KEY)
        resposta = requests.get(url)
        data = resposta.json()
        if data.get("articles"):
            for article in data["articles"]:
                # a ordem de prioridade para o texto que será analisado é: descrição da notícia - conteúdo - título
                text = article.get("description") or article.get("content") or article.get("title")
                if text:
                        corpus.append({
                        "title": article.get("title"),
                        "text": text,
                        "source": article.get("source", {}).get("name", "N/A"),
                        "url": article.get("url")
                    })

    print(f"Notícias obtidas ({len(corpus)}):")
    for i, news in enumerate(corpus):
        # excluir o [...] padrão presente no final do content
        clean_text = news['text'].replace('[...]', '').strip()
        print(f"--- Notícia {i+1} ---\nFonte: {news['source']}\nTítulo: {news['title']}\nResumo: {clean_text}\n")
        news['text'] = clean_text
    return corpus

In [ ]:
# chamada para obter o corpus
news_corpus_comparacao = get_news(query="trump")

Digite sua Chave da News API: ··········
Notícias obtidas (90):
--- Notícia 1 ---
Fonte: Fox News
Título: Fox News Channel’s Aishah Hasnie promoted to anchor, White House correspondent
Resumo: Aishah Hasnie has been promoted to anchor and White House correspondent, FOX News Media President and Executive Editor Jay Wallace announced on Tuesday.

--- Notícia 2 ---
Fonte: Fox News
Título: Scandal-plagued journalist Olivia Nuzzi's book gets brutal reviews, called disappointing and 'forgettable'
Resumo: Olivia Nuzzi's new book "American Canto" faced criticism from major publications on Tuesday, with reviewers calling it "disappointing," poorly written and incoherent.

--- Notícia 3 ---
Fonte: Fox News
Título: Americans must pay higher fee for a ticket to the world’s most visited museum
Resumo: The Louvre is raising ticket prices for non-EU visitors by $12 per person following the October heist of jewels worth $102 million. The museum is also implementing new security measures.

--- Notícia 

In [ ]:
import os
import requests
import pandas as pd
from getpass import getpass
from IPython.display import display, Markdown
from typing import List

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [ ]:
# chave api

if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = getpass("Digite sua Chave da API do Gemini: ") 

GEMINI_KEY = os.environ.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    google_api_key=GEMINI_KEY
)
print("\nLLM Gemini 2.5 Flash inicializaado")

Digite sua Chave da API do Gemini: ··········

LLM Gemini 2.5 Flash inicializaado


In [ ]:
# estrutura de saída
class AnalisePolitica(BaseModel):
    viés_político: str = Field(description="Classificação única do viés político: ESQUERDA, DIREITA ou NEUTRO.")
    palavras_chave: List[str] = Field(description="Lista das 5 principais palavras-chave ou frases-chave do texto.")
    entidades_ner: List[str] = Field(description="Lista de PESSOAS e ORGANIZAÇÕES nomeadas.")

json_parser = JsonOutputParser(pydantic_object=AnalisePolitica)

# prompt único
unificado_prompt = PromptTemplate(
    input_variables=["text"],
    template="""Você é um analista político de texto neutro. Sua tarefa é analisar o viés, palavras-chave e entidades do texto a seguir, e retornar a análise estritamente em formato JSON.

    1. Viés Político: Classifique o viés implícito em uma única palavra: ESQUERDA, DIREITA ou NEUTRO. Para isto, considere o lado em que a notícia é prejudicial,
    exemplo: se a notícia é prejudicial para a imagem da esquerda, considere de direita.
    2. Palavras-chave: Liste as 5 frases-chave mais relevantes.
    3. Entidades NER: Liste todas as pessoas e organizações mencionadas. Se não houver, use 'Nenhuma'.

    Texto da Notícia: {text}

    {format_instructions}
    """,
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

unificada_chain_lcel = unificado_prompt | llm | json_parser

In [ ]:
try:
    df_results = pd.read_csv(RESULT_FILE_NAME)
    processed_urls = set(df_results['URL'].unique())
    print(f"Resultados anteriores carregados ({len(df_results)} artigos).")
except FileNotFoundError:
    processed_urls = set()
    print("Ficheiro de resultados não encontrado. Iniciando um novo.")

# Execução do LLM
print("\nIniciando análise\n")

new_results = []
request_counter = 0
MAX_REQUISICOES = 250

for i, news in enumerate(news_corpus_comparacao):

    if request_counter >= MAX_REQUISICOES:
        print(f"\nLIMITE DIÁRIO DE REQUISIÇÕES ATINGIDO")
        break

    if news['url'] in processed_urls:
        continue

    if not news['text']:
        continue

    print(f"Processando Notícia {i+1} ({news['source']}) | Requisições feitas: {request_counter+1}...")

    input_data = {"text": news['text']}

    try:
        analise_json = unificada_chain_lcel.invoke(input_data)
        request_counter += 1

        current_result = {
            "Título": news['title'],
            "Veículo": news['source'],
            "URL": news['url'],
            "Viés Político": analise_json.get('viés_político', 'N/A').strip(),
            "Palavras-chave": ", ".join(analise_json.get('palavras_chave', [])),
            "Entidades (NER)": ", ".join(analise_json.get('entidades_ner', []))
        }
        new_results.append(current_result)

        if request_counter % 10 == 0:
            pd.DataFrame(new_results).to_csv(RESULT_FILE_NAME, mode='a', index=False, header=(not os.path.exists(RESULT_FILE_NAME)))
            new_results = []
            print(f"Progresso salvo. Requisições feitas nesta sessão: {request_counter}")

    except Exception as e:
        print(f"Erro Crítico ao processar notícia {i+1}: {e}. Salve o progresso e reinicie.")
        break

if new_results:
    pd.DataFrame(new_results).to_csv(RESULT_FILE_NAME, mode='a', index=False, header=(not os.path.exists(RESULT_FILE_NAME)))


Ficheiro de resultados não encontrado. Iniciando um novo.

Iniciando análise

Processando Notícia 1 (Fox News) | Requisições feitas: 1...
Processando Notícia 2 (Fox News) | Requisições feitas: 2...
Processando Notícia 3 (Fox News) | Requisições feitas: 3...
Processando Notícia 4 (Fox News) | Requisições feitas: 4...
Processando Notícia 5 (Fox News) | Requisições feitas: 5...
Processando Notícia 6 (Fox News) | Requisições feitas: 6...
Processando Notícia 7 (Fox News) | Requisições feitas: 7...
Processando Notícia 8 (Fox News) | Requisições feitas: 8...
Processando Notícia 9 (Fox News) | Requisições feitas: 9...
Processando Notícia 10 (Fox News) | Requisições feitas: 10...
Progresso salvo. Requisições feitas nesta sessão: 10
Processando Notícia 11 (Fox News) | Requisições feitas: 11...


* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 42.515832398s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, retry_delay {
  seconds: 42
}
].


Processando Notícia 12 (Fox News) | Requisições feitas: 12...


* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 40.396570082s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, retry_delay {
  seconds: 40
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 36.260955798s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.

Processando Notícia 13 (Fox News) | Requisições feitas: 13...
Processando Notícia 14 (Fox News) | Requisições feitas: 14...
Processando Notícia 15 (Fox News) | Requisições feitas: 15...
Processando Notícia 16 (Fox News) | Requisições feitas: 16...
Processando Notícia 17 (Fox News) | Requisições feitas: 17...
Processando Notícia 18 (Fox News) | Requisições feitas: 18...
Processando Notícia 19 (Fox News) | Requisições feitas: 19...
Processando Notícia 20 (Fox News) | Requisições feitas: 20...
Progresso salvo. Requisições feitas nesta sessão: 20
Processando Notícia 21 (Fox News) | Requisições feitas: 21...
Processando Notícia 22 (Fox News) | Requisições feitas: 22...
Processando Notícia 23 (Fox News) | Requisições feitas: 23...
Processando Notícia 24 (Fox News) | Requisições feitas: 24...
Processando Notícia 25 (Fox News) | Requisições feitas: 25...
Processando Notícia 26 (Fox News) | Requisições feitas: 26...
Processando Notícia 27 (Fox News) | Requisições feitas: 27...
Processando Notíc

...

Processando Notícia 55 (BBC News) | Requisições feitas: 55...
Processando Notícia 56 (BBC News) | Requisições feitas: 56...
Processando Notícia 57 (BBC News) | Requisições feitas: 57...
Processando Notícia 58 (BBC News) | Requisições feitas: 58...
Processando Notícia 59 (BBC News) | Requisições feitas: 59...
Processando Notícia 60 (BBC News) | Requisições feitas: 60...
Progresso salvo. Requisições feitas nesta sessão: 60
Processando Notícia 61 (MSNBC) | Requisições feitas: 61...
Processando Notícia 62 (MSNBC) | Requisições feitas: 62...
Processando Notícia 63 (MSNBC) | Requisições feitas: 63...
Processando Notícia 64 (MSNBC) | Requisições feitas: 64...
Processando Notícia 65 (MSNBC) | Requisições feitas: 65...
Processando Notícia 66 (MSNBC) | Requisições feitas: 66...
Processando Notícia 67 (MSNBC) | Requisições feitas: 67...
Processando Notícia 68 (MSNBC) | Requisições feitas: 68...
Processando Notícia 69 (MSNBC) | Requisições feitas: 69...
Processando Notícia 70 (MSNBC) | Requisições

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 30.184372358s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, retry_delay {
  seconds: 30
}
].


Processando Notícia 74 (MSNBC) | Requisições feitas: 74...


* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 28.071726593s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, retry_delay {
  seconds: 28
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 23.956642273s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.

Processando Notícia 75 (MSNBC) | Requisições feitas: 75...
Processando Notícia 76 (MSNBC) | Requisições feitas: 76...
Processando Notícia 77 (MSNBC) | Requisições feitas: 77...
Processando Notícia 78 (MSNBC) | Requisições feitas: 78...
Processando Notícia 79 (MSNBC) | Requisições feitas: 79...
Processando Notícia 80 (MSNBC) | Requisições feitas: 80...
Progresso salvo. Requisições feitas nesta sessão: 80
Processando Notícia 81 (MSNBC) | Requisições feitas: 81...
Processando Notícia 82 (MSNBC) | Requisições feitas: 82...
Processando Notícia 83 (MSNBC) | Requisições feitas: 83...
Processando Notícia 84 (MSNBC) | Requisições feitas: 84...
Processando Notícia 85 (MSNBC) | Requisições feitas: 85...
Processando Notícia 86 (MSNBC) | Requisições feitas: 86...
Processando Notícia 87 (MSNBC) | Requisições feitas: 87...
Processando Notícia 88 (MSNBC) | Requisições feitas: 88...
Processando Notícia 89 (MSNBC) | Requisições feitas: 89...
Processando Notícia 90 (MSNBC) | Requisições feitas: 90...
Pro

In [ ]:
# exibição do resultado
try:
    df_final = pd.read_csv(RESULT_FILE_NAME)
    display(Markdown(f"## Resultados Finais Agregados ({len(df_final)} Artigos Processados no Total)"))
    df_final = df_final[['Título', 'Veículo', 'Viés Político', 'Palavras-chave', 'Entidades (NER)', 'URL']]
    display(df_final.head(10))

    display(Markdown("### Distribuição de Viés por Veículo"))
    viés_summary = df_final.groupby(['Veículo', 'Viés Político']).size().unstack(fill_value=0)
    display(viés_summary)

except FileNotFoundError:
    print("\nNenhum artigo processado com sucesso.")

## Resultados Finais Agregados (90 Artigos Processados no Total)

,Título,Veículo,Viés Político,Palavras-chave,Entidades (NER),URL
0,Fox News Channel’s Aishah Hasnie promoted to a...,Fox News,NEUTRO,"Aishah Hasnie promoted, anchor and White House...","Aishah Hasnie, Jay Wallace, FOX News Media, Wh...",https://www.foxnews.com/media/fox-news-channel...
1,Scandal-plagued journalist Olivia Nuzzi's book...,Fox News,NEUTRO,"Olivia Nuzzi's new book, ""American Canto"", fac...",Olivia Nuzzi,https://www.foxnews.com/media/scandal-plagued-...
2,Americans must pay higher fee for a ticket to ...,Fox News,NEUTRO,"Louvre, raising ticket prices, non-EU visitors...",The Louvre,https://www.foxnews.com/travel/americans-pay-h...
3,"Giving Tuesday reminds us that all gifts, smal...",Fox News,NEUTRO,"Giving Tuesday, post-Thanksgiving gratitude, g...",Nenhuma,https://www.foxnews.com/opinion/giving-tuesday...
4,Afghan evacuee arrested before DC shooting fed...,Fox News,DIREITA,"Mohammad Dawood Alokozay, federal charges, thr...","Mohammad Dawood Alokozay, Biden",https://www.foxnews.com/politics/afghan-evacue...
5,Stefanik accuses Johnson of 'siding with' Demo...,Fox News,ESQUERDA,"Rep. Elise Stefanik, Speaker Mike Johnson, sid...","Elise Stefanik, Mike Johnson, Democrats, Repub...",https://www.foxnews.com/politics/stefanik-accu...
6,"Top Dem blasts Trump's cartel strikes, says ad...",Fox News,ESQUERDA,"Jamie Raskin blasts Trump, Trump's Caribbean n...","Jamie Raskin, Trump",https://www.foxnews.com/politics/top-dem-blast...
7,Hegseth backs special ops chief's 'combat deci...,Fox News,NEUTRO,"Pete Hegseth defends military commander, Congr...","Pete Hegseth, Congress",https://www.foxnews.com/politics/hegseth-backs...
8,"Mace campaign staffer quits governor bid team,...",Fox News,NEUTRO,"Nancy Mace's campaign consultant, J. Austin Mc...","Nancy Mace, J. Austin McCubbin, Rand Paul, Pre...",https://www.foxnews.com/politics/mace-campaign...
9,Pete Hegseth touts higher physical standards f...,Fox News,DIREITA,"new military physical standards, highest male ...","Pete Hegseth, Katie Miller",https://www.foxnews.com/media/pete-hegseth-tou...


### Distribuição de Viés por Veículo

Viés Político,DIREITA,ESQUERDA,NEUTRO
Veículo,,,
BBC News,4,8,18
Fox News,10,6,14
MSNBC,3,19,8


In [ ]:
import pandas as pd
dados = {
    'Modelo': ['GEMINI 2.5-FLASH', 'CHATGPT', 'GEMINI'],
    'Porcentagem': ['65,09%', '82,84%', '81,07%']
}

# Criar o DataFrame (a tabela)
df = pd.DataFrame(dados)

# Exibir a tabela
print("Comparação da assertividade dos modelos")
print(df.to_markdown(index=False))

Comparação da assertividade dos modelos
| Modelo           | Porcentagem   |
|:-----------------|:--------------|
| GEMINI 2.5-FLASH | 65,09%        |
| CHATGPT          | 82,84%        |
| GEMINI           | 81,07%        |
